In [2]:
from transformers import AutoTokenizer,AutoModel

/opt/anaconda3/envs/llamafactory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained("/Users/zhangyf/llm/bert-base-chinese")
model = AutoModel.from_pretrained("/Users/zhangyf/llm/bert-base-chinese")

In [21]:
tokens = tokenizer.tokenize("我爱自然语言处理")
print( tokens)

['我', '爱', '自', '然', '语', '言', '处', '理']


In [22]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2769, 4263, 5632, 4197, 6427, 6241, 1905, 4415]


In [23]:
tokens = tokenizer.convert_ids_to_tokens(ids)
print(tokens)

['我', '爱', '自', '然', '语', '言', '处', '理']


In [24]:
ids = tokenizer.encode("我爱自然语言处理", max_length=20, truncation=True, padding="max_length")
print(ids)

[101, 2769, 4263, 5632, 4197, 6427, 6241, 1905, 4415, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
string = tokenizer.decode(ids)
print(string)

[CLS] 我 爱 自 然 语 言 处 理 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [26]:
data = tokenizer("我爱自然语言处理",max_length=20,truncation=True,padding="max_length")
print(data)

{'input_ids': [101, 2769, 4263, 5632, 4197, 6427, 6241, 1905, 4415, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
texts = ["我爱自然语言处理", "我爱人工智能", "我们一起学习"]
encoded = tokenizer(texts, max_length=10, truncation=True, padding="max_length", return_tensors="pt")
print(encoded)

{'input_ids': tensor([[ 101, 2769, 4263, 5632, 4197, 6427, 6241, 1905, 4415,  102],
        [ 101, 2769, 4263,  782, 2339, 3255, 5543,  102,    0,    0],
        [ 101, 2769,  812,  671, 6629, 2110,  739,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}


In [25]:
import torch

with torch.no_grad():
    outputs = model(
        **encoded
    )
print(outputs.keys())
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)
print(outputs.pooler_output)
print(outputs.last_hidden_state[:,0,:])

odict_keys(['last_hidden_state', 'pooler_output'])
torch.Size([3, 10, 768])
torch.Size([3, 768])
tensor([[ 0.9990,  0.9994,  0.9955,  ..., -0.9990, -0.9827,  0.9323],
        [ 0.9972,  1.0000,  0.7786,  ..., -0.9976, -0.9421,  0.8830],
        [ 0.9995,  1.0000,  0.9952,  ..., -0.9994, -0.9972,  0.9198]])
tensor([[-0.3187, -0.1426, -0.2135,  ...,  0.2972,  0.2283, -0.2938],
        [-0.2295,  0.1760, -0.2449,  ...,  0.1220,  0.4493,  0.1019],
        [-0.1337,  0.3413,  0.1714,  ...,  0.6724,  0.1278,  0.0090]])


### datasets库

In [9]:
from datasets import load_dataset

In [30]:
ds_dict = load_dataset("csv", data_files="../review_analyze_bert/data/raw/online_shopping_10_cats.csv")
print(ds_dict)

DatasetDict({
    train: Dataset({
        features: ['cat', 'label', 'review'],
        num_rows: 62774
    })
})


In [31]:
train_datasets = ds_dict["train"]
print(train_datasets)

Dataset({
    features: ['cat', 'label', 'review'],
    num_rows: 62774
})


In [32]:
print(train_datasets[0])

{'cat': '书籍', 'label': 1, 'review': '做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一颗年轻的心。我想，这是他能很好的和孩子沟通的一个重要因素。读刘墉的文章，总能让我看到一个快乐的平易近人的父亲，他始终站在和孩子同样的高度，给孩子创造着一个充满爱和自由的生活环境。很喜欢刘墉在字里行间流露出的做父母的那种小狡黠，让人总是忍俊不禁，父母和子女之间有时候也是一种战斗，武力争斗过于低级了，智力较量才更有趣味。所以，做父母的得加把劲了，老思想老观念注定会一败涂地，生命不息，学习不止。家庭教育，真的是乐在其中。'}


In [33]:
print(train_datasets[:3])

{'cat': ['书籍', '书籍', '书籍'], 'label': [1, 1, 1], 'review': ['做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一颗年轻的心。我想，这是他能很好的和孩子沟通的一个重要因素。读刘墉的文章，总能让我看到一个快乐的平易近人的父亲，他始终站在和孩子同样的高度，给孩子创造着一个充满爱和自由的生活环境。很喜欢刘墉在字里行间流露出的做父母的那种小狡黠，让人总是忍俊不禁，父母和子女之间有时候也是一种战斗，武力争斗过于低级了，智力较量才更有趣味。所以，做父母的得加把劲了，老思想老观念注定会一败涂地，生命不息，学习不止。家庭教育，真的是乐在其中。', '作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理的火花。整本书的结构颇有特点，从当时（本书写于八十年代）流行的计算机话题引入，再用数学、物理学、宇宙学做必要的铺垫——这些内容占据了大部分篇幅，最后回到关键问题：电脑能不能代替人脑。和现在流行的观点相反，作者认为人的某种“洞察”是不能被算法模拟的。也许作者想说，人的灵魂是无可取代的。', '作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？为什么在文化上有着深刻纽带关系的中国和日本却在经济发展上有着极大的差异？为什么英国的北美殖民地造就了经济强大的美国，而西班牙的北美殖民却造就了范后的墨西哥？……很有价值，但不包括【中国近代史专业】。']}


In [34]:
print(train_datasets[0:3]["review"])

['做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一颗年轻的心。我想，这是他能很好的和孩子沟通的一个重要因素。读刘墉的文章，总能让我看到一个快乐的平易近人的父亲，他始终站在和孩子同样的高度，给孩子创造着一个充满爱和自由的生活环境。很喜欢刘墉在字里行间流露出的做父母的那种小狡黠，让人总是忍俊不禁，父母和子女之间有时候也是一种战斗，武力争斗过于低级了，智力较量才更有趣味。所以，做父母的得加把劲了，老思想老观念注定会一败涂地，生命不息，学习不止。家庭教育，真的是乐在其中。', '作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理的火花。整本书的结构颇有特点，从当时（本书写于八十年代）流行的计算机话题引入，再用数学、物理学、宇宙学做必要的铺垫——这些内容占据了大部分篇幅，最后回到关键问题：电脑能不能代替人脑。和现在流行的观点相反，作者认为人的某种“洞察”是不能被算法模拟的。也许作者想说，人的灵魂是无可取代的。', '作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？为什么在文化上有着深刻纽带关系的中国和日本却在经济发展上有着极大的差异？为什么英国的北美殖民地造就了经济强大的美国，而西班牙的北美殖民却造就了范后的墨西哥？……很有价值，但不包括【中国近代史专业】。']


In [35]:
# 删除列
train_datasets = train_datasets.remove_columns(["cat"])
print(train_datasets)

Dataset({
    features: ['label', 'review'],
    num_rows: 62774
})


In [37]:
# 过滤
train_datasets = train_datasets.filter(lambda x: x["review"] is not None and x["review"].strip() != "" and x["label"] in [0,1])
print(train_datasets)

Filter: 100%|██████████| 62774/62774 [00:00<00:00, 705825.12 examples/s]

Dataset({
    features: ['label', 'review'],
    num_rows: 62773
})


In [57]:
# 划分数据集
dataset_dict = train_datasets.train_test_split(test_size=0.2)
print(dataset_dict["train"])
print(dataset_dict["test"][0])

Dataset({
    features: ['label', 'review'],
    num_rows: 50218
})
{'label': 1, 'review': '自己亲手装的。有工具不难装：还可以。家庭用买小了。送的花洒不怎么样。'}


In [51]:
# 编码数据
tokenizer = AutoTokenizer.from_pretrained("/Users/zhangyf/llm/bert-base-chinese")
# 逐个样本处理
def encode_fn(example):
    return tokenizer(example["review"], truncation=True, padding="max_length", max_length=10)

dataset = dataset_dict["train"].map(encode_fn, batched=False,remove_columns=["review"])
print(dataset)

Map: 100%|██████████| 50218/50218 [00:08<00:00, 5790.18 examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50218
})


In [56]:
def batch_fn(examples):
    return tokenizer(examples["review"], truncation=True, padding="max_length", max_length=10)
dataset = dataset_dict["train"].map(batch_fn, batched=True,remove_columns=["review"])
print(dataset)

Map: 100%|██████████| 50218/50218 [00:01<00:00, 39328.52 examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50218
})


In [58]:
dataset_dict = dataset_dict.map(batch_fn,batched=True,remove_columns=["review"])
print(dataset_dict)

Map: 100%|██████████| 12555/12555 [00:00<00:00, 33113.28 examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50218
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12555
    })
})


In [59]:
dataset_dict.save_to_disk("data")

Saving the dataset (1/1 shards): 100%|██████████| 12555/12555 [00:00<00:00, 1988726.41 examples/s]


In [61]:
from datasets import load_from_disk

dataset_dict = load_from_disk("data")
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50218
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12555
    })
})


In [63]:
dataset_dict["train"].to_csv("data/train.csv")

Creating CSV from Arrow format: 100%|██████████| 51/51 [00:01<00:00, 25.57ba/s]


4923850

In [64]:
dataset_dict["test"].to_json("data/test.jsonl")

Creating json from Arrow format: 100%|██████████| 13/13 [00:00<00:00, 285.07ba/s]


1887941

In [71]:
dataset = load_dataset("json", data_files="data/test.jsonl")
train_datasets = dataset["train"]
print(train_datasets)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12555
})


### pytorch DataLoader继承

In [74]:
train_datasets.set_format(type="torch")
print(train_datasets)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12555
})


In [79]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_datasets, batch_size=128, shuffle=True)
for batch in train_loader:
    print(batch["input_ids"].shape)
    print(batch["attention_mask"].shape)
    print(batch["token_type_ids"].shape)
    print(batch["label"].shape)
    break

torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128])
